# Introduction to bps_to_omop

This document serves as a first step to introduce the rationale behind this package.

## 2. Generation of OMOP tables

Once the original files are "omopized", we can start building the OMOP tables themselves. Depending on the original data, not every table needs to be built. This generates a situation where a general procedure to build every table is not optimal, hence, we have provided single script for each table that rely on bps_to_omop modules. Which tables to build depends almost entirely on the specifics of the original data.

Take into accoutn that, tables CDM_SOURCE, PERSON and OBSERVATION_PERIOD are always mandatory.

but our main goal is to create the tables PERSON, CONDITION_OCCURRENCE and MEASUREMENT. The process is summarized in the following diagram.

```{mermaid}
%%{init: {'flowchart': {'curve': 'linear'}} }%%
flowchart LR
	script_cdm_source["genomop_cdm_source"]
	script_person["genomop_person"]
	script_obs["genomop_observation_period"]
	script_visit["genomop_visit_occurrence"]
	script_cond["genomop_condition_occurrence"]
	script_meas["genomop_measurement"]

	folder_cdm_source{{<i>/omop_intermediate/CDM_SOURCE/</i>}}
	folder_person{{<i>/omop_intermediate/PERSON/</i>}}
	folder_obs{{<i>/omop_intermediate/OBSERVATION_PERIOD/</i>}}
	folder_visit{{<i>/omop_intermediate/VISIT_OCCURRENCE/</i>}}
	folder_cond{{<i>/omop_intermediate/CONDITION_OCCURRENCE/</i>}}
	folder_meas{{<i>/omop_intermediate/MEASUREMENT/</i>}}

	subgraph group2 [Independent processes]
		script_cdm_source	
		script_person
		script_obs
		script_visit
	end
	
	script_visit --> script_cond
	script_visit --> script_meas

	script_cdm_source --> folder_cdm_source
	script_person --> folder_person
	script_obs --> folder_obs
	script_visit --> folder_visit
	script_cond --> folder_cond
	script_meas --> folder_meas

	subgraph group3 [OMOP Tables]
		folder_cdm_source
		folder_person 
		folder_obs 
		folder_visit  
		folder_cond
		folder_meas
	end
```